# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [90]:
# Import packages
import re
import Bio.SeqIO
import Bio.Seq

# Define gene ends as variables 
HA_end = "CCGGATTTGCATATAATGATGCACCAT"
NA_end = "CACGATAGATAAATAATAGTGCACCAT"

def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence read matching UPSTREAM-BARCODE in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    List or None
        Sequence of the barcode in the forward orientation and the gene id in a list, or `None` if no match to expected barcoded sequence. 
    """
    # Convert to forward orientation
    seq_fwd = seqread.reverse_complement()

    # Change seq_fwd to a string
    seq_fwd_str = str(seq_fwd)
    
    # Create a pattern to match the upstream sequence and barcode
    pattern = re.compile(f"(?<={upstream})([ATCG]{{{bclen}}})")

    # Search for the pattern in the forward sequence
    match = pattern.search(str(seq_fwd_str))

    # Check if you get a match. If there's no match, its an invalid sequence
    if match: 
        # Define the barcode
        barcode = match.group(0)
        # determine if the sequence is HA and NA. return the barcode with the gene id
        if HA_end in seq_fwd_str:
            return [barcode, 'HA']
        elif NA_end in seq_fwd_str:
            return [barcode, 'NA']
    # For the invalid sequences, determine if it is HA or Na
    else: 
        if HA_end in seq_fwd_str:
            return ['invalid_barcode', 'HA']
        elif NA_end in seq_fwd_str:
            return ['invalid_barcode', 'NA']    
    return None

# Read sequences from the barcodes_R1.fastq file and create a list of only the sequences (as Seq objects)
# Just copied from lecture 09

seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))
seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def count_barcodes(seqreads_Seq, bclen, upstream='AGGCGGCCGC'):
    """
    Counts occurrences of barcodes with a known upstream sequence.

    Parameters
    ----------
    seqreads_Seq : list of Seq objects
        List of nucleotide sequence reads to analyze.
    bclen : int
        Length of barcode.
    upstream : str
        Sequence upstream of the barcode.

    Returns
    -------
    A list containing: 
        dict
            Dictionary with HA barcodes as keys and their counts as values.
        dict
            Dictionary with NA barcodes as keys and their counts as values. 
        int
            Number of invalid sequences that also don't match a gene 
    """
    # Initialize an empty dictionaries to store barcode counts for ha and na
    ha_barcode_counts = {}
    na_barcode_counts = {}
    # Counter for sequences without a valid barcode
    invalid_count = 0
    # Iterate over each sequence
    for seqread in seqreads_Seq:
        # Create a variable that can be used to check if read_barcodes gives None
        rb_result = read_barcode(seqread, bclen, upstream=upstream)
        if rb_result:
            # determine barcode and gene type 
            barcode, gene_type = rb_result
            # Check if the gene type is HA or NA. If it's one of these, increment the gene type
            #  barcode count or set to 1 if not in dictionary.
            if gene_type == 'HA':
                ha_barcode_counts[barcode] = ha_barcode_counts.get(barcode, 0) + 1
            elif gene_type == 'NA':
                na_barcode_counts[barcode] = na_barcode_counts.get(barcode, 0) + 1
        else:
            # Increment the invalid count if no barcode is found
            invalid_count += 1
    return [ha_barcode_counts, na_barcode_counts, invalid_count]

# Run the barcode counting function for each gene
ha_counts = count_barcodes(seqreads_Seq, 16)[0]
na_counts = count_barcodes(seqreads_Seq, 16)[1]

# Make a function to count the total reads of a barcode by iterating over all the barcodes and adding their count to count 
def total_reads(gene_counts):
    count = 0
    for barcode in gene_counts:
        count += gene_counts[barcode]
    return count

# Print out results
print(str(total_reads(ha_counts)) + ' sequences map to HA')
print(str(total_reads(na_counts)) + ' sequences map to NA')



5409 sequences map to HA
4122 sequences map to NA


2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [91]:
# Print out results
print('There were', ha_counts['invalid_barcode'], 'invalid HA sequences')
print('There were', na_counts['invalid_barcode'], 'invalid NA sequences')

There were 160 invalid HA sequences
There were 213 invalid NA sequences


3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [92]:
# Make a function to find the barcode with the most counts and output the counts
def find_most_counts (gene_counts):
    # Initialize the highest barcode as an empty string  
    highest_barcode = ''
    # Set the highest count to a value below any of the values in the dictionary
    highest_count = -10
    # Iterate over all the elements in gene_counts 
    for barcode in gene_counts:
        # Check if you're looking at the invalid barcode key
        if barcode == 'invalid_barcode':
            None
        # If its not the invalid key, check if the barcode count is higher than the current highest count. If it is, replace the current highest with barcode count 
        else:
            barcode_count = gene_counts[barcode]
            if barcode_count > highest_count:
                highest_count = barcode_count
                highest_barcode = barcode
    # Return a list containing the highest barcode and the highest count 
    return [highest_barcode, highest_count]
# Print out the results
print('The HA barcode with the most counts is', find_most_counts(ha_counts)[0], 'with', find_most_counts(ha_counts)[1], 'counts')
print('The NA barcode with the most counts is', find_most_counts(na_counts)[0], 'with', find_most_counts(na_counts)[1], 'counts')

The HA barcode with the most counts is CCCGACCCGACATTAA with 155 counts
The NA barcode with the most counts is ACCAGTTCTCCCCGGG with 152 counts
